In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score

df = pd.read_csv('data/titanic.csv')
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
M = 'survived ~ pclass + sex + sibsp + parch + fare + C(embarked) '
columns = ['survived', 'pclass','sex','sibsp','parch','fare','embarked']
df = df.dropna(subset = columns)

In [5]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [10]:
print("mean fare: {:.2f}".format(df.fare.mean()))
df[df.fare > 512]

mean fare: 33.22


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
49,1,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C,3,NaN,"Austria-Hungary / Germantown, Philadelphia, PA"
50,1,1,"Cardeza, Mrs. James Warburton Martinez (Charlo...",female,58.0,0,1,PC 17755,512.3292,B51 B53 B55,C,3,NaN,"Germantown, Philadelphia, PA"
183,1,1,"Lesurer, Mr. Gustave J",male,35.0,0,0,PC 17755,512.3292,B101,C,3,NaN,NaN
302,1,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,NaN,C,3,NaN,NaN


In [12]:
seeds = [1,8,17]
for seed in seeds:
    # create the train and test subset
    np.random.seed(seed)
    train_index = df.sample(frac = 0.7).index
    train = df.loc[df.index.isin(train_index)]
    test = df.loc[~df.index.isin(train_index)]
    # train the model
    results = smf.logit(M, train).fit()
    yhat    = results.predict(test)
    # AUC score
    auc = roc_auc_score(test['survived'], yhat)
    print(" seed: {} AUC: {:.2f}  ".format(seed, auc))

Optimization terminated successfully.
         Current function value: 0.458138
         Iterations 6
 seed: 1 AUC: 0.80  
Optimization terminated successfully.
         Current function value: 0.466795
         Iterations 6
 seed: 8 AUC: 0.83  
Optimization terminated successfully.
         Current function value: 0.488925
         Iterations 6
 seed: 17 AUC: 0.86  


In [31]:
df = pd.read_csv('data/credit_default.csv')
# Convert Yes No to 1/0
df.loc[df['default'] == 'No', 'default'] = 0
df.loc[df['default'] == 'Yes', 'default'] = 1
df["default"]=df.default.astype(int)


dtype('int32')

In [32]:
# # Fit a logistic regression model
results = smf.logit('default ~ student + balance + income', df).fit()
yhat = results.predict(df)


Optimization terminated successfully.
         Current function value: 0.078577
         Iterations 10


In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df['default'], yhat > 0.5)

array([[9627,   40],
       [ 228,  105]], dtype=int64)